In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import os
import datetime
import numpy as np
from random import random, randint
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import load_img,  img_to_array, array_to_img
from numpy import load, expand_dims

In [ ]:
def generateRealSamples(dataset, LSamples, patchShape):
    ix = np.random.randint(0, dataset.shape[0], LSamples)
    X = dataset[ix]
    y = np.ones((LSamples, patchShape, patchShape, 1))
    return X, y

In [ ]:
def generateFakeSamples(generatorModel, dataset, patchShape):
    X = generatorModel.predict(dataset)
    y = np.zeros((len(X), patchShape, patchShape, 1))
    return X, y

In [ ]:
def updateImagePool(pool, images, max_size=50):
    selected = list()
    for image in images:
        if len(pool) < max_size:
            pool.append(image)
            selected.append(image)
        elif random() < 0.5:
            selected.append(image)
        else:
            ix = randint(0, len(pool) - 1)
            selected.append(pool[ix])
            pool[ix] = image
    return np.asarray(selected)

In [ ]:
def summarizePerformance(LStep, generatorModel, trainX, name, LSamples=5):
    InputX, _ = generateRealSamples(trainX, LSamples, 0)
    OutputX, _ = generateFakeSamples(generatorModel, InputX, 0)
    InputX = (InputX + 1) / 2.0
    OutputX = (OutputX + 1) / 2.0
    for i in range(LSamples):
        plt.subplot(2, LSamples, 1 + i)
        plt.axis('off')
        plt.imshow(InputX[i])
    for i in range(LSamples):
        plt.subplot(2, LSamples, 1 + LSamples + i)
        plt.axis('off')
        plt.imshow(OutputX[i])
    filename = 'plots/%s_generatedPlot_%06d.png' % (name, (LStep+1))
    plt.savefig(filename)
    plt.close()

In [ ]:
def loadImage(filename, size=(256,256)):
    image = load_img(filename, target_size=size)
    image = img_to_array(image)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image = expand_dims(image, 0)
    return image